# Postfix To NFA

In [2]:
import json
from collections import defaultdict
import shunting_yard

In [3]:
epsilon = "\u03B5"
alphanumerics = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

class State:
    def __init__(self,name):
        self.name = name
        self.out_edges = []
        self.in_edges = []

class Edge:
    def __init__(self, label:str ,dest:State):
        self.label = label
        self.dest = dest

class NFA:
    def __init__(self, char: str, start: State, accepting: State):
        self.states = [start,accepting]
        self.start = start
        self.accepting = accepting

        # edge = Edge(char,self.accepting)
        # self.start.out_edges.append(edge)
        # self.accepting.in_edges.append(edge)

    @staticmethod
    def postfix2NFA(postfix:str):
        stack = []
        state_counter= 1

        for char in postfix:
            if char=='.':
                nfa2 = stack.pop()
                nfa1 = stack.pop()

                edge = Edge(epsilon,nfa2.start)
                nfa1.accepting.out_edges.append(edge)
                nfa2.start.in_edges.append(edge)

                concatenated_nfa = NFA(epsilon, nfa1.start, nfa2.accepting)

                concatenated_nfa.states = nfa1.states + nfa2.states
                stack.append(concatenated_nfa)

            elif char=='|':
                nfa2 = stack.pop()
                nfa1 = stack.pop()
                new_start = State('S' + str(state_counter))
                new_accept = State('S' +  str(state_counter +1))
                state_counter += 2
                edge1,edge2,edge3,edge4 = Edge(epsilon,nfa1.start),Edge(epsilon,nfa2.start),Edge(epsilon,new_accept),Edge(epsilon,new_accept)

                new_start.out_edges.append(edge1)
                nfa1.start.in_edges.append(edge1)

                new_start.out_edges.append(edge2)
                nfa2.start.in_edges.append(edge2)

                nfa1.accepting.out_edges.append(edge3)
                new_accept.in_edges.append(edge3)

                nfa2.accepting.out_edges.append(edge4)
                new_accept.in_edges.append(edge4)

                ored_nfa= NFA(epsilon, new_start, new_accept)

                ored_nfa.states = [new_start, new_accept] + nfa1.states + nfa2.states
                stack.append(ored_nfa)

            elif char=='*':
                nfa = stack.pop()
                new_start = State('S' + str(state_counter))
                new_accept = State('S' +  str(state_counter +1))
                state_counter += 2
                edge1,edge2,edge3,edge4 = Edge(epsilon,nfa.start),Edge(epsilon,new_accept),Edge(epsilon,new_start),Edge(epsilon,new_accept)

                new_start.out_edges.append(edge1)
                nfa.start.in_edges.append(edge1)

                nfa.accepting.out_edges.append(edge2)
                new_accept.in_edges.append(edge2)

                nfa.accepting.out_edges.append(edge3)
                new_start.in_edges.append(edge3)

                new_start.out_edges.append(edge4)
                new_accept.in_edges.append(edge4)

                zero_or_more_nfa = NFA(epsilon, new_start, new_accept)
                zero_or_more_nfa.states = [new_start, new_accept] + nfa.states
                stack.append(zero_or_more_nfa)

            elif char=='+':
                nfa = stack.pop()
                new_start = State('S' + str(state_counter))
                new_accept = State('S' +  str(state_counter +1))
                state_counter += 2
                edge1,edge2,edge3 = Edge(epsilon,nfa.start),Edge(epsilon,new_accept),Edge(epsilon,new_start)

                new_start.out_edges.append(edge1)
                nfa.start.in_edges.append(edge1)

                nfa.accepting.out_edges.append(edge2)
                new_accept.in_edges.append(edge2)

                nfa.accepting.out_edges.append(edge3)
                new_start.in_edges.append(edge3)

                one_or_more_nfa = NFA(epsilon, new_start, new_accept)
                one_or_more_nfa.states = [new_start, new_accept] + nfa.states
                stack.append(one_or_more_nfa)

            elif char == '?':
                nfa = stack.pop()
                new_start = State('S' + str(state_counter))
                new_accept = State('S' +  str(state_counter +1))
                state_counter += 2
                edge1,edge2,edge3 = Edge(epsilon,nfa.start),Edge(epsilon,new_accept),Edge(epsilon,new_accept)

                new_start.out_edges.append(edge1)
                nfa.start.in_edges.append(edge1)

                nfa.accepting.out_edges.append(edge2)
                new_accept.in_edges.append(edge2)

                new_start.out_edges.append(edge3)
                new_accept.in_edges.append(edge3)

                zero_or_one_nfa = NFA(epsilon, new_start, new_accept)
                zero_or_one_nfa.states = [new_start, new_accept] + nfa.states
                stack.append(zero_or_one_nfa)

            elif char in alphanumerics:
                start_state = State('S' + str(state_counter))
                accept_state = State('S' +  str(state_counter +1))
                state_counter += 2

                edge = Edge(char,accept_state)

                start_state.out_edges.append(edge)
                accept_state.in_edges.append(edge)

                single_char_nfa = NFA(char, start_state, accept_state)
                single_char_nfa.states.extend([start_state,accept_state])

                stack.append(single_char_nfa)

        return stack.pop()

    def to_json(self):
        nfa_dict = {"startingState": self.start.name}

        for state in self.states:
            state_info = {"isTerminatingState": state == self.accepting}
            # edge_dict = {}
            for edge in state.out_edges:
                # destination = None
                # for s in self.states:
                #     if edge in s.in_edges:
                #         destination = s
                #         break
                destination = edge.dest

                if edge.label in state_info:
                  # if destination.name not in edge_dict[edge.label]:
                    # edge_dict[edge.label].append(destination.name)
                   state_info[edge.label] = list(state_info[edge.label]) + [destination.name]
                else:
                    state_info[edge.label] = [destination.name]

            # for label, destinations in edge_dict.items():
            #     state_info[label] = destinations

            nfa_dict[state.name] = state_info

        with open("NFA.json", "w",encoding="utf-8") as json_file:
            json.dump(nfa_dict, json_file, indent=4, ensure_ascii=False)

        return nfa_dict


In [4]:
postfix = shunting_yard.infix2postfix('a+b')
# nfa = NFA.postfix2NFA(postfix='A+B*.?CD|.')
# nfa = NFA.postfix2NFA(postfix='ab|c|d|e|fA.|B|C|')
nfa = NFA.postfix2NFA(postfix)
# nfa = NFA.postfix2NFA(postfix='ab|c|d|e|f0.|1|2|3|4|5|6|7|8|9|3.2.')
nfa.to_json()

{'startingState': 'S3',
 'S3': {'isTerminatingState': False, 'ε': ['S1']},
 'S4': {'isTerminatingState': False, 'ε': ['S5']},
 'S1': {'isTerminatingState': False, 'a': ['S2']},
 'S2': {'isTerminatingState': False, 'ε': ['S4', 'S3']},
 'S5': {'isTerminatingState': False, 'b': ['S6']},
 'S6': {'isTerminatingState': True}}

In [5]:
from graphviz import Digraph

def visualize_NFA(nfa):

  gra = Digraph(graph_attr={'rankdir':'LR'})


  for s in nfa.states:
      if(s.name == nfa.start.name):
        gra.node("", _attributes={'shape' : 'none'})
        gra.edge("", s.name)
      if(s.name == nfa.accepting.name):
        gra.node(s.name, _attributes={'peripheries' : '2'})
      else:
        gra.node(s.name)

  edge_set = set()
  for state in nfa.states:
    for edge in state.out_edges:

        destination = edge.dest
        edge_key = (state.name, destination.name, edge.label)
        if edge_key not in edge_set:
            edge_set.add(edge_key)
            gra.edge(state.name, destination.name, label=edge.label)

  gra.format = 'png'
  gra.render('NFA', view = True)
  return gra.source

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
from IPython.display import Image
visualize_NFA(nfa)
Image(filename='NFA.png')

NameError: name 'visualize_NFA' is not defined

# NFA To DFA

In [ ]:
class DFA:
    def __init__(self):
        self.states = [] #supersets
        self.start = None
        self.accepting = []
        self.alphabets = []
        

    def set_alphabets(self,nfa:NFA):
        temp_alphabets = set()
        for state in nfa.states:
            for edge in state.out_edges:
                if edge.label != epsilon:
                    temp_alphabets.add(edge.label)

        self.alphabets = list(temp_alphabets)

    def epsilon_closure(self,states:list[State]):
        stack = states
        closure = set(states)
        
        while stack:
            s = stack.pop()
            for edge in s.out_edges:
                if edge.label == epsilon and edge.dest not in closure:
                    closure.add(edge.dest)
                    stack.append(edge.dest)
        return list(closure)

    
    def move(self,states:list[State], char:str):
        destinations = []
        for s in states:
            for edge in s.out_edges:
                if edge.label == char:
                    destinations.append(edge.dest)
        return destinations
    
    
    def remove_duplicates(self,states:list[State]):
        unique_states = []
        seen_names = set()  
        for state in states:
            if state.name not in seen_names:
                seen_names.add(state.name)  
                unique_states.append(state)  
        return unique_states 
    
    def rename_states(self,transitions):
        renamed_transitions = {}
        states_map={}
        i=0
        for state in transitions:
            if state == "startingState":
                continue
            states_map[state] = "S"+str(i)
            i+=1
        
        for state in transitions:
            if state == "startingState":
                renamed_transitions["startingState"] = states_map[transitions["startingState"]]
                continue
            renamed_transitions[states_map[state]] = {}
            for edge in transitions[state]:
                if edge == "isTerminatingState" :
                    renamed_transitions[states_map[state]]["isTerminatingState"] = transitions[state]["isTerminatingState"]
                    continue
                renamed_transitions[states_map[state]][edge] = states_map[transitions[state][edge]]
        
        return renamed_transitions,states_map
    
    
    
    def NFA2DFA(self,nfa:NFA):
        self.set_alphabets(nfa)
        start = self.epsilon_closure([nfa.start])
        self.states.append(start)
        self.start = start
        
        transitions ={}
        start_string = " ".join(sorted([s.name for s in self.start]))
        transitions["startingState"]= start_string

        unvisited = [start]
        while unvisited:
            state_list = unvisited.pop()
            for alphabet in self.alphabets:
                destinations = self.move(state_list,alphabet)
                if not destinations:
                    continue
                closure = self.epsilon_closure(destinations)

            
                for s in closure:
                    if s == nfa.accepting:
                        self.accepting.append(closure)
                        break

                if closure not in self.states:
                    self.states.append(closure)
                    unvisited.append(closure)
                
                from_state_string = " ".join(sorted([s.name for s in state_list]))
                to_state_string = " ".join(sorted([s.name for s in closure]))
            
                
                if from_state_string not in transitions:
                    transitions[from_state_string] = {}

                transitions[from_state_string][alphabet] = to_state_string
                transitions[from_state_string]["isTerminatingState"] = state_list in self.accepting

  
        for state in self.accepting:
            state_string=" ".join(sorted([s.name for s in state]))
            if state_string not in transitions:
                transitions[state_string] = {}
                transitions[state_string]["isTerminatingState"]=True
                

        renamed_transitions,states_map = self.rename_states(transitions)
        for i,state in enumerate(self.states):
            self.states[i] = State(states_map[" ".join(sorted([s.name for s in state]))])
        
        for i,state in enumerate(self.accepting):
            self.accepting[i] = State(states_map[" ".join(sorted([s.name for s in state]))])
            
        self.states = self.remove_duplicates(self.states)
        self.accepting = self.remove_duplicates(self.accepting)
        
        
        
        return transitions,renamed_transitions
    
            
            
    def to_json(self,transitions:dict,renamed_transitions:dict):
        with open("DFA_transitions.json", "w") as json_file:
            json.dump(transitions, json_file, indent=4, ensure_ascii=False)
        
        with open("DFA_transitions_renamed.json", "w") as json_file:
            json.dump(renamed_transitions, json_file, indent=4, ensure_ascii=False)


In [7]:
postfix = shunting_yard.infix2postfix('a+|b+')
nfa = NFA.postfix2NFA(postfix)
nfa.to_json()
dfa = DFA()
transitions,renamed_transitions=dfa.NFA2DFA(nfa)
dfa.to_json(transitions,renamed_transitions)



In [134]:
class MinimizedDFA:
    def __init__(self, dfa: DFA, transitions: dict):
        self.dfa = dfa
        self.transitions = transitions
        self.minimized_transitions = {}
        
    def __difference(self, states_a: list[State], states_b: list[State]):
        names_b = {state.name for state in states_b}  
        return [state for state in states_a if state.name not in names_b]  
    
    def __split(self,group: list[State],partition: list[list[State]]): 
    # If all states in the group go to the same subgroup for all inputs, don't split
    # Otherwise, split into subgroups where states have identical transition behavior

        subgroups = {}  # key: tuple of destination groups, value: list of states
                        # ex: {(0, 1): [A, B], (1, 0): [C, D]}

        for state in group:
            destination_groups = []

            for input in self.dfa.alphabets:
                next_state = self.transitions.get(state.name, {}).get(input, None)
                destination_groups.append(next_state)
                for i, existing_group in enumerate(partition):
                    if next_state in existing_group:
                        destination_groups.append(i)
                        break
            
            
            if tuple(destination_groups) not in subgroups:
                subgroups[tuple(destination_groups)] = []
            subgroups[tuple(destination_groups)].append(state)

        # return true if a split happens, false if no split
        if len(subgroups) > 1:
            return subgroups.values(),True
        return [group],False

            
    def minimize(self):
        # inital partition TT
        non_accepting = self.__difference(self.dfa.states,self.dfa.accepting) # {A, B, C, D}  non accepting states
        accepting = self.dfa.accepting # {E}  accepting states

        partitions = [non_accepting, accepting] #TTnew =TT = {A, B, C, D} {E}
        # partition is a list of lists(groups) 
        changed = True
        while changed:
            changed = False
            new_partitions = []

            for group in partitions:  
                subgroups, is_split = self.__split(group, partitions)
                if is_split:
                    changed = True
                new_partitions.extend(subgroups)

            partitions = new_partitions
            
                
        state_map = {}  
        minimized_states = []
        minimized_accepting = []
        minimized_transitions = {}

        # rename groups (to be one state) and check on accepting states
        for i,group in enumerate(partitions):
            new_state = f"S{i}"
            minimized_states.append(new_state)
            state_map[frozenset(group)] = new_state

            if any(state in self.dfa.accepting for state in group):
                minimized_accepting.append(new_state)

        
        # build transitions dict after minimization
        for group in partitions:
            from_state = state_map[frozenset(group)]
            minimized_transitions[from_state] = {}
            
            sample_state = group[0]

            for input in self.dfa.alphabets:
                next_state = self.transitions.get(sample_state.name, {}).get(input, None)

                if next_state:
                    for group2 in partitions:
                        for state in group2:
                            if next_state == state.name :
                                minimized_transitions[from_state][input] = state_map[frozenset(group2)]
                                break


        # add accept states
        for state in minimized_states:
            if state in minimized_accepting:
                minimized_transitions[state]["isTerminatingState"] = True
            else:
                minimized_transitions[state]["isTerminatingState"] = False
        
        # add start state
        # for group in partitions:
        #     if self.dfa.start in group:
        #         print(f"start state{self.dfa.start} at group {group}")
        #         minimized_transitions["startingState"] = state_map[frozenset(group)]
        #         break

        minimized_transitions["startingState"] = self.transitions["startingState"]
                    
        self.dfa.states = minimized_states
        self.dfa.accepting = minimized_accepting
        self.minimized_transitions = minimized_transitions

        return minimized_transitions

    def to_json(self):
        with open("MinimizedDFA.json", "w") as json_file:
            json.dump(self.minimized_transitions, json_file, indent=4, ensure_ascii=False)


In [135]:
postfix = shunting_yard.infix2postfix('a+b')
nfa = NFA.postfix2NFA(postfix)
nfa.to_json()
dfa = DFA()
transitions,renamed_transitions=dfa.NFA2DFA(nfa)
dfa.to_json(transitions,renamed_transitions)
minimized_dfa = MinimizedDFA(dfa, renamed_transitions)
minimized_dfa.minimize()
minimized_dfa.to_json()
